# Documents Summarizer

In [ ]:
from crewai import Crew, Agent, Task, LLM, Process
from crewai_tools import FileReadTool
import os
# Set up API key
os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

### LLMs

In [3]:
summarizer_llm = LLM(
    model="gpt-4",
    temperature=0.7,  # Balanced creativity for summarization
    max_tokens=500    # Limit the output length
)

manager_llm = LLM(
    model="gpt-4",
    temperature=0.3,  # Factual and deterministic
    max_tokens=200    # Short feedback or approval responses
)

### Tools

In [4]:
file_read_tool = FileReadTool()

### Agents

In [6]:
# Summarizer Agent
summarizer = Agent(
    role="Text Summarizer",
    goal="Summarize the text in the file {path_to_file} into a concise and coherent summary.",
    backstory="A highly skilled summarizer specialized in condensing long texts.",
    tools=[file_read_tool],
    llm=summarizer_llm,
    verbose=True,
)

# Manager Agent
manager = Agent(
    role="Content Quality Manager",
    goal="Ensure that the summary provided is realistic and meets quality standards.",
    backstory="An experienced manager skilled in evaluating text summaries.",
    llm=manager_llm,
    verbose=True,
)

### Tasks

In [7]:
# Summarizer Task
summarize_task = Task(
    name="Summarize Text Task",
    agent=summarizer,
    description="Read the given text file and generate a concise summary.",
    expected_output="A brief and coherent summary of the text."
)

# Manager Task
validate_task = Task(
    name="Validate Summary Task",
    agent=manager,
    description=(
        "Evaluate the summary for realism and coherence. "
        "If it lacks realism or clarity, request a revision from the Summarizer."
    ),
    expected_output="Approval or specific feedback for improvement."
)

### Crew

In [8]:
crew = Crew(
    agents=[summarizer],
    tasks=[summarize_task],
    process=Process.hierarchical,
    manager_agent=manager
)

### Main

In [ ]:
result = crew.kickoff(inputs={"path_to_file": "src/sample_text.txt"})
print("Final Output:")
print(result)

# Agent: Content Quality Manager
## Task: Read the given text file and generate a concise summary.
